In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split # Used if you were splitting here

In [23]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print("Training Data Shape:", df_train.shape)
print("Testing Data Shape:", df_test.shape)


Training Data Shape: (1200, 81)
Testing Data Shape: (260, 80)


In [24]:
# Save 'Id' columns for submission
test_ID = df_test['Id']

In [25]:
# Drop the original 'Id' column from the dataframes
df_train.drop("Id", axis=1, inplace=True)
df_test.drop("Id", axis=1, inplace=True)


In [26]:

missing_counts = df_train.isna().sum().astype(np.int64)
print(missing_counts)

PropertyClass         0
ZoningCategory        0
RoadAccessLength    223
LandArea              0
RoadType              0
                   ... 
MonthSold             0
YearSold              0
DealType              0
DealCondition         0
HotelValue            0
Length: 80, dtype: int64


Clearing the columns with many NaN values

In [27]:

missing_counts = (
    df_train[['PoolQuality', 'ExtraFacility', 'ServiceLaneType', 'BoundaryFence','LoungeQuality','FacadeType']]
    .isna()
    .sum()
    .astype(np.int64)
)

print(missing_counts)

PoolQuality        1194
ExtraFacility      1154
ServiceLaneType    1125
BoundaryFence       963
LoungeQuality       560
FacadeType          702
dtype: int64


In [28]:
missing_counts = (
    df_test[['PoolQuality', 'ExtraFacility', 'ServiceLaneType', 'BoundaryFence','LoungeQuality','FacadeType']]
    .isna()
    .sum()
    .astype(np.int64)
)

print(missing_counts)

PoolQuality        259
ExtraFacility      252
ServiceLaneType    244
BoundaryFence      216
LoungeQuality      130
FacadeType         170
dtype: int64


In [29]:

cols_to_drop = [
    'PoolQuality', 'ExtraFacility', 'ServiceLaneType', 
    'BoundaryFence', 'FacadeType','LoungeQuality'
]
df_train = df_train.drop(columns=cols_to_drop)
df_test = df_test.drop(columns=cols_to_drop)


For some columns very less amount of data is missing so its better to remove those rows as it will confuse our model

In [30]:

print(df_train[['ElectricalSystem']].isna().sum().astype(np.int64))
print(df_test[['ElectricalSystem']].isna().sum().astype(np.int64))


ElectricalSystem    1
dtype: int64
ElectricalSystem    0
dtype: int64


In [31]:
df_train.dropna(subset=['ElectricalSystem'], inplace=True)

Impute NaN values with 'None' for features where missing means absence

In [32]:
# Basement-related columns
basement_cols = [
    'BasementHeight', 'BasementCondition', 'BasementExposure', 
    'BasementFacilityType1', 'BasementFacilityType2'
]
df_train[basement_cols] = df_train[basement_cols].fillna('None')
df_test[basement_cols] = df_test[basement_cols].fillna('None')

# Garage/Parking related columns (excluding ParkingConstructionYear)
parking_cat_cols = [
    'ParkingType', 'ParkingFinish', 'ParkingQuality', 
    'ParkingCondition'
]
df_train[parking_cat_cols] = df_train[parking_cat_cols].fillna('None')
df_test[parking_cat_cols] = df_test[parking_cat_cols].fillna('None')


Checking for duplicates

In [33]:
print(df_train.duplicated().sum())
print(df_test.duplicated().sum())

0
0


Possible incosistencies:


RenovationYear < ConstructionYear

YearSold < ConstructionYear


In [34]:
df_cleaned1 = df_train[
    (df_train['RenovationYear'] >= df_train['ConstructionYear']) &
    (df_train['YearSold'] >= df_train['ConstructionYear'])
].copy()

print("Original rows:", len(df_train))
print("Cleaned rows:", len(df_cleaned1))
print("Rows removed:", len(df_train) - len(df_cleaned1))

df_train = df_cleaned1


df_cleaned2 = df_test[
    (df_test['RenovationYear'] >= df_test['ConstructionYear']) &
    (df_test['YearSold'] >= df_test['ConstructionYear'])
].copy()

print("Original rows:", len(df_test))
print("Cleaned rows:", len(df_cleaned2))
print("Rows removed:", len(df_test) - len(df_cleaned2))

df_test = df_cleaned2

Original rows: 1199
Cleaned rows: 1199
Rows removed: 0
Original rows: 260
Cleaned rows: 260
Rows removed: 0


Creating temporal features and removing original features

In [35]:

df_train['TotalOutdoorArea'] = (df_train['TerraceArea'] + df_train['OpenVerandaArea'] + df_train['EnclosedVerandaArea'] + df_train['ScreenPorchArea']).fillna(0)
df_train['TotalSF'] = (df_train['GroundFloorArea'] + df_train['UpperFloorArea'] + df_train['ParkingArea'] + df_train['TotalOutdoorArea']).fillna(0)

df_train['TotalBaths'] = (df_train['FullBaths'] + 0.5 * df_train['HalfBaths'] +
                    df_train['BasementFullBaths'] + 0.5 * df_train['BasementHalfBaths']).fillna(0)

df_train['OverallScore'] = (df_train['OverallQuality'] + df_train['OverallCondition']) / 2.0 # Assumes these columns were NOT dropped

# --- 2. Temporal Features ---
df_train['Age'] = df_train['YearSold'] - df_train['ConstructionYear']
df_train['YearsSinceRemodel'] = df_train['YearSold'] - df_train['RenovationYear']
df_train['YearsSinceRemodel'] = np.where(df_train['YearsSinceRemodel'] < 0, 0, df_train['YearsSinceRemodel'])
df_train.loc[df_train['RenovationYear'] == df_train['ConstructionYear'], 'YearsSinceRemodel'] = df_train['Age']

# --- 3. Interaction Feature (Example) ---
df_train['Qual_x_GroundSF'] = df_train['OverallQuality'] * df_train['GroundFloorArea'] # Assumes these columns were NOT dropped

# --- 4. Feature Reduction/Drop ---
drop_cols = ['GroundFloorArea', 'UpperFloorArea', 
                'ConstructionYear', 'RenovationYear', 
                'FullBaths', 'HalfBaths','ParkingArea',
                'BasementFullBaths', 'BasementHalfBaths', 'BasementFacilitySF1', 'BasementFacilitySF2', 
                'TerraceArea', 'OpenVerandaArea','EnclosedVerandaArea', 'ScreenPorchArea','OverallQuality', 'OverallCondition', 'SeasonalPorchArea']


In [36]:

df_test['TotalOutdoorArea'] = (df_test['TerraceArea'] + df_test['OpenVerandaArea'] + df_test['EnclosedVerandaArea'] + df_test['ScreenPorchArea']).fillna(0)
df_test['TotalSF'] = (df_test['GroundFloorArea'] + df_test['UpperFloorArea'] + df_test['ParkingArea'] + df_test['TotalOutdoorArea']).fillna(0)

df_test['TotalBaths'] = (df_test['FullBaths'] + 0.5 * df_test['HalfBaths'] +
                    df_test['BasementFullBaths'] + 0.5 * df_test['BasementHalfBaths']).fillna(0)

df_test['OverallScore'] = (df_test['OverallQuality'] + df_test['OverallCondition']) / 2.0 # Assumes these columns were NOT dropped

# --- 2. Temporal Features ---
df_test['Age'] = df_test['YearSold'] - df_test['ConstructionYear']
df_test['YearsSinceRemodel'] = df_test['YearSold'] - df_test['RenovationYear']
df_test['YearsSinceRemodel'] = np.where(df_test['YearsSinceRemodel'] < 0, 0, df_test['YearsSinceRemodel'])
df_test.loc[df_test['RenovationYear'] == df_test['ConstructionYear'], 'YearsSinceRemodel'] = df_test['Age']

# --- 3. Interaction Feature (Example) ---
df_test['Qual_x_GroundSF'] = df_test['OverallQuality'] * df_test['GroundFloorArea'] # Assumes these columns were NOT dropped

# --- 4. Feature Reduction/Drop ---
drop_cols = ['GroundFloorArea', 'UpperFloorArea', 
                'ConstructionYear', 'RenovationYear', 
                'FullBaths', 'HalfBaths','ParkingArea',
                'BasementFullBaths', 'BasementHalfBaths', 'BasementFacilitySF1', 'BasementFacilitySF2', 
                'TerraceArea', 'OpenVerandaArea','EnclosedVerandaArea', 'ScreenPorchArea','OverallQuality', 'OverallCondition', 'SeasonalPorchArea']
df_train.drop(columns=drop_cols, inplace=True)
df_test.drop(columns=drop_cols, inplace=True)


In [37]:
median_facade_area = df_train['FacadeArea'].median()
df_train['FacadeArea'].fillna(median_facade_area, inplace=True)

median_road_access = df_train['RoadAccessLength'].median()
df_train['RoadAccessLength'].fillna(median_road_access, inplace=True)

/tmp/ipykernel_26708/1916592955.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train['FacadeArea'].fillna(median_facade_area, inplace=True)
/tmp/ipykernel_26708/1916592955.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [38]:
median_facade_area = df_test['FacadeArea'].median()
df_test['FacadeArea'].fillna(median_facade_area, inplace=True)

median_road_access = df_test['RoadAccessLength'].median()
df_test['RoadAccessLength'].fillna(median_road_access, inplace=True)

/tmp/ipykernel_26708/2669533664.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test['FacadeArea'].fillna(median_facade_area, inplace=True)
/tmp/ipykernel_26708/2669533664.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

Check for outliers

In [39]:
outlier_features = ['TotalSF', 'LandArea']
target_col = 'HotelValue'

plt.figure(figsize=(12, 5))

for i, col in enumerate(outlier_features):
    plt.subplot(1, 2, i + 1)
    # Generate scatter plot
    sns.scatterplot(x=df_train[col], y=df_train[target_col])
    plt.title(f'{col} vs. {target_col}', fontsize=12)
    plt.xlabel(col)
    plt.ylabel(target_col)

plt.tight_layout()
plt.savefig('outlier_scatterplots_V2.png')
plt.close()

Removing the outliers

In [40]:
# 1. Define columns for IQR analysis
iqr_cols = ['TotalSF', 'LandArea'] 

# Initialize an index set for IQR outliers to drop
outlier_iqr_indices = pd.Index([])

for col in iqr_cols:
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df_train[col].quantile(0.25)
    Q3 = df_train[col].quantile(0.75)
    
    # Calculate IQR
    IQR = Q3 - Q1
    
    # Define the outlier bounds (Q1 - 1.5*IQR and Q3 + 1.5*IQR)
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Find indices outside these IQR-based bounds
    out_of_range_indices = df_train[(df_train[col] < lower_bound) | (df_train[col] > upper_bound)].index
    
    # Add these new outlier indices to the overall set
    outlier_iqr_indices = outlier_iqr_indices.union(out_of_range_indices)

print(f"IQR-based removal identified {len(outlier_iqr_indices)} total rows to drop.")


# --- 5. Consolidate ALL Outliers and Execute Final Drop ---

# Outliers previously identified by manual checks (assuming these lines are run earlier)
outlier_tsf_indices = df_train[(df_train['TotalSF'] > 6000) & (df_train['HotelValue'] < 12.5)].index
outlier_land_indices = df_train[(df_train['LandArea'] > 50000) & (df_train['HotelValue'] < 13)].index

# Combine ALL outliers (manual checks and IQR checks)
outliers_to_drop = outlier_tsf_indices.union(outlier_land_indices).union(outlier_iqr_indices)

# Execute the drop operation
df_train.drop(outliers_to_drop, inplace=True)

print(f"\nTotal unique rows dropped from all checks: {len(outliers_to_drop)}")
print(f"Remaining training rows: {df_train.shape[0]}")




IQR-based removal identified 71 total rows to drop.

Total unique rows dropped from all checks: 71
Remaining training rows: 1128


Pre Processed CSVs

In [41]:
df_train.to_csv('preprocesssed_train_V2.csv',index=False)
df_test.to_csv('preprocesssed_test_V2.csv',index=False)

In [42]:
# Load your currently saved processed data
X_train = pd.read_csv('preprocesssed_train_V2.csv')
X_test = pd.read_csv('preprocesssed_test_V2.csv')

# --- 1. Separate Target and Features ---
# Assume 'HotelValue_Log' is the log-transformed target you created
y_train = X_train['HotelValue']
X_train.drop(columns=['HotelValue'], inplace=True, errors='ignore')
X_test.drop(columns=['HotelValue'], inplace=True, errors='ignore') # Test set usually has no target

# --- 2. Categorical Encoding (One-Hot) ---
# Identify all remaining text columns
nominal_cols = X_train.select_dtypes(include='object').columns.tolist()

# Combine for unified One-Hot Encoding (Crucial for consistent columns)
combined_df = pd.concat([X_train, X_test], axis=0)

# Apply One-Hot Encoding
combined_encoded = pd.get_dummies(combined_df, columns=nominal_cols, drop_first=True)

# Separate back into encoded train and test sets
X_train_encoded = combined_encoded.iloc[:len(X_train)]
X_test_encoded = combined_encoded.iloc[len(X_train):]

# --- 3. Feature Scaling (Fit on Train, Transform on Test) ---
scaler = StandardScaler()

# FIT and TRANSFORM the training data
X_train_scaled = scaler.fit_transform(X_train_encoded)
# ONLY TRANSFORM the test data using the parameters learned from the training data
X_test_scaled = scaler.transform(X_test_encoded)

# Convert back to DataFrames
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_encoded.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_encoded.columns)

# Reattach the target to the training data
X_train_scaled_df['HotelValue'] = y_train.values 

# --- 4. Final Save ---
X_train_scaled_df.to_csv('final_processed_train_V2.csv', index=False)
X_test_scaled_df.to_csv('final_processed_test_V2.csv', index=False)

